# Jármű felismerés YOLOv3 segítségével

Az alábbi fájl beolvas egy képet, és a yolov3-at használva felismeri a legnagyobb talált objektumot. Mivel különböző cimkével rendelkeznek a járművek, ezért a legnagyobb terület alapján választja ki a keresett járművünket, ez azért működhet, mert a képek többségében majdnem csak a keresett jármű szerepel.

Súlyok letöltése: https://pjreddie.com/media/files/yolov3.weights -> car_recognition mappába kell rakni

Forrás: https://towardsdatascience.com/yolo-object-detection-with-opencv-and-python-21e50ac599e9

Vannak olyan esetek, ahol nem járművön található a rendszám pl.: 18248688.jpg, ebben az esetben kihagyható a jármű felismerés, és a rendszámmal kezdhető.

### Háló inicializálása

In [50]:
# Library-k beimportálása után konstansokba felvesszük a config fájl útját, a súlyokat és a cimkéket.
import cv2
import numpy as np

IMAGE_URL = '../dataset/17873500.jpg'
CONFIG_FILE = './yolov3.cfg'
WEIGHTS = './yolov3.weights'
CLASSES = './yolov3.txt'

# Beolvassuk a képet, elmentjük a magasságát és a szélességét, majd a scale faktort ami 1/255-nek felel meg. (0..255) -> (0..1)
image = cv2.imread(IMAGE_URL)
Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392 # 1/255

# Beolvassuk a címkéket a fájlból, és elmentjük.
classes = None
with open(CLASSES, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Létrehozzuk az előtanított hálót a súly és a config alapján, majd átalakítjuk a képet és beadjuk bemenetnek a hálónak.
net = cv2.dnn.readNet(WEIGHTS, CONFIG_FILE)
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
net.setInput(blob)

### Kimenetek kinyerése a háló segítségével

In [51]:
# Következtetéseket lefuttatjuk a hálón keresztül és az előrejelzéseket kigyűjtjük a kimeneti rétegekből
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
outs = net.forward(output_layers)

# Változók inicializálás
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

# Kimenetek feldolgozása
for out in outs:
    # detection első 4 paramétere: (középpont) x, y, szélesség, magasság
    for detection in out:
        scores = detection[5:] # Cimkék valószínűségei kigyűjtése
        class_id = np.argmax(scores) # Max valószínűség kiválasztása
        confidence = scores[class_id]
        if confidence > 0.5:
            # Középpontból kiszámoljuk a bal felső sarkot, és visszaskálázzuk a 0..1-ről a 0..H és 0..W-re az értékeket.
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2 
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

# Non-max suppression használata a kimenetre
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

### Kimenetek feldolgozása

In [52]:
# Kiválasztjuk a legnagyobb területtel rendelkező találatot a képről
max_i = -1
max_area = 0
for i in indices:
    box = boxes[i]
    current_area = box[2] * box[3]
    if max_area < current_area:
        max_area = current_area
        max_i = i

# Ha nincs találat kiírjuk, különben levágjuk a találatra a képet és megnyitjuk.
if max_i == -1:
    print("Car not found!")
else:
    biggest_box = boxes[max_i]
    x = int(biggest_box[0])
    y = int(biggest_box[1])
    w = int(biggest_box[2])
    h = int(biggest_box[3])
    car_image = image[y:y+h,x:x+w]

    cv2.imshow("Car detection", car_image)
    cv2.waitKey()
    cv2.destroyAllWindows()